In [5]:
from nlp_chat_bot.rag import RAG
from nlp_chat_bot.model.late_chunking_embedding import LateChunkingEmbedding
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
dataset_path = "../data"
model_download_path = "../models"
vector_store_path = "../chromadb"
embedding_function = LateChunkingEmbedding(model_download_path)

llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
rag = RAG(dataset_path, embedding_function, vector_store_path, late_chunking=True, llm=llm_gemini)
print("LENGTH", rag.get_num_docs())
docs_retrieved = rag.retrieve(state = {"question": "What is my conclusion in my project report on image inpainting?", "context": []})

print("Num docs:", len(docs_retrieved["context"]))

for i in range(len(docs_retrieved["context"])):
    doc = docs_retrieved["context"][i]
    print("\n\n", "#"*30,"\n")
    print(f"doc {i}: (score: {doc.metadata['score']})")
    print(doc.page_content)

d:\Programs\Anaconda\envs\nlp_project_chatbot\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  7.76it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  1.67it/s]
0it [00:00, ?it/s]


Embedding and storing 6 documents...


100%|██████████| 6/6 [00:21<00:00,  3.51s/it]


LENGTH 144
Num docs: 3


 ############################## 

doc 0: (score: 39.987790474214655)
Image Inpainting with Basic Convolutional Networks
Robin Meneust, Ethan Pinto
December 2024
1 Introduction
In the context of our ”AI-Based Image Processing”
course, we worked on this project, in which we repro-
duced and tested a specific image inpainting approach,
defined by the paper ”Context Encoders: Feature Learn-
ing by Inpainting”(Pathak et al., 2016)[1].
Image inpainting consists of filling hole(s) in an im-
age. There exist different methods to do so (e.g. they
compared their results with Photoshop). In this paper,
they used a context encoder trained in an adversarial
way. Basically there is a generator, this is our context
encoder (here an encoder and a decoder) that given an
image of size 128x128 with a dropout region (a ”hole”,
with values set to 0) tries to predict what should be inside
the hole. We focused on the simplest case here for the
dropout region: a square in the center o

In [8]:
rag.invoke(query={"question":"What is my conclusion in my project report on image inpainting?"})

{'question': 'What is my conclusion in my project report on image inpainting?',
 'context': (Document(metadata={'score': 39.987790474214655}, page_content='Image Inpainting with Basic Convolutional Networks\nRobin Meneust, Ethan Pinto\nDecember 2024\n1 Introduction\nIn the context of our ”AI-Based Image Processing”\ncourse, we worked on this project, in which we repro-\nduced and tested a specific image inpainting approach,\ndefined by the paper ”Context Encoders: Feature Learn-\ning by Inpainting”(Pathak et al., 2016)[1].\nImage inpainting consists of filling hole(s) in an im-\nage. There exist different methods to do so (e.g. they\ncompared their results with Photoshop). In this paper,\nthey used a context encoder trained in an adversarial\nway. Basically there is a generator, this is our context\nencoder (here an encoder and a decoder) that given an\nimage of size 128x128 with a dropout region (a ”hole”,\nwith values set to 0) tries to predict what should be inside\nthe hole. We foc